# Speech Emotion Recognition - Transformer

## Import necessary libraries

In [9]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import IPython
import librosa # librosa is a Python library for analyzing audio and music used to extract the data from audio files it later
import librosa.display
from IPython.display import Audio # to play the audio files
plt.style.use('seaborn-white')
from python_speech_features import mfcc

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from os.path import dirname, join as pjoin
from scipy.io import wavfile
import scipy.io

from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout, Conv1D, MaxPooling1D, BatchNormalization
from keras.layers import concatenate
from keras.models import Model
from keras.layers.normalization import layer_normalization
from tensorflow.keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import LSTM
from keras.applications.vgg19 import VGG19
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.utils import np_utils, to_categorical

import torchaudio
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel
import speech_recognition as sr

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

In [2]:
Tess = 'C:/Users/TestUser/OneDrive/Documents/Data Science Capstone/Dataset/TESS Toronto emotional speech set data'

In [3]:
path = []

emotion = []

directory = os.listdir(Tess)

In [4]:
for i in directory:
    
    folder = os.listdir(Tess + '/' + i)      
    
    for k in folder:
        
        if i == 'OAF_angry' or i == 'YAF_angry':
            emotion.append('Angry')
        elif i == 'OAF_disgust' or i == 'YAF_disgust':
            emotion.append('Disgust')
        elif i == 'OAF_Fear' or i == 'YAF_fear':
            emotion.append('Fear')
        elif i == 'OAF_happy' or i == 'YAF_happy':
            emotion.append('Happy')
        elif i == 'OAF_neutral' or i == 'YAF_neutral':
            emotion.append('Neutral')                                
        elif i == 'OAF_Pleasant_surprise' or i == 'YAF_pleasant_surprised':
            emotion.append('Surprise')               
        elif i == 'OAF_Sad' or i == 'YAF_sad':
            emotion.append('Sad')
        else:
            emotion.append('Unknown')
        path.append(Tess + "/" + i + "/" + k)

In [5]:
Tess_df = pd.DataFrame(emotion, columns = ['Emotions'])

Tess_df = pd.concat([Tess_df, pd.DataFrame(path, columns = ['Path'])], axis = 1)

In [6]:
emotion = 'Angry'

path = np.array(Tess_df['Path'][Tess_df['Emotions'] == emotion])[0]

array, sampling_rate = librosa.load(path)

IPython.display.Audio(path)

In [14]:
print(torch.__version__)
print(torchaudio.__version__)

torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

2.0.0+cpu
2.0.1+cpu
cpu


In [16]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H

In [19]:
model = bundle.get_model().to(device)

In [20]:
wave, sample_rate = torchaudio.load(path)

wave = wave.to(device)

if sample_rate != bundle.sample_rate:
    
    wave = torchaudio.functional.resample(wave, sample_rate, bundle.sample_rate)

In [21]:
with torch.inference_mode():
    
    emission, _ = model(wave)

In [29]:
class Decoder(torch.nn.Module):
    
    def __init__(self, labels, blank = 0):
        
        super().__init__()
        
        self.labels = labels
        
        self.blank = blank

        # Given a sequence emission over labels, get the best path string
    def forward(self, emission: torch.Tensor) -> str:

        indices = torch.argmax(emission, dim = -1) 
        
        indices = torch.unique_consecutive(indices, dim = -1)
        
        indices = [i for i in indices if i != self.blank]
        
        # The resulting transcript
        return "".join([self.labels[i] for i in indices])   

In [30]:
decoder = Decoder(labels = bundle.get_labels())

transcript = decoder(emission[0])

In [31]:
print(transcript)

IPython.display.Audio(path)

SAY|THE|WORD|BACK|
